# Purpose of notebook: see tokenization and truncation for data

In [11]:
%pip install transformers
%pip install pandas

import transformers, pandas as pd

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [43]:
# read csv
file_path = "/work/LauraSørineVoldgaard#8191/data/all_articles_allsides_abortion.csv"
articles = pd.read_csv(file_path)
articles

,textID,words,date,country,source,url,headline,body,leaning
0,91719925,723.0,23-11-02,US,The Boston Globe,https://www.bostonglobe.com/2023/11/02/metro/d...,this band plans to drown out the men's march a...,"kirk israel, an activist musician and member o...",Left
1,91721731,1248.0,23-11-03,US,Fox News,https://www.foxnews.com/politics/virginias-ele...,virginia's elections a key 2024 barometer and ...,youngkin wore a similar red vest two years ago...,Right
2,103177268,1947.0,23-11-03,US,inquisitr.com,https://www.theatlantic.com/ideas/archive/2023...,here's what biden can do to change his grim po...,"whatever your theory, it should take into acco...",Left
3,103177271,1513.0,23-11-03,US,inquisitr.com,https://www.theatlantic.com/ideas/archive/2023...,don't equate anti-zionism with anti-semitism,"on october 7, the islamist militant group hama...",Left
4,103177279,2607.0,23-11-03,US,inquisitr.com,https://www.theatlantic.com/books/archive/2023...,do you have free will?,writing a review is an exercise in free will. ...,Left
...,...,...,...,...,...,...,...,...,...
8090,97824446,1244.0,23-03-28,US,bostonglobe.com,https://bostonglobe.com/2023/03/26/metro/rev-e...,"rev. elinor lockwood yeo, reproductive rights ...","rev. elinor lockwood yeo of newton, a reproduc...",Left
8091,97847400,648.0,23-03-29,US,sfchronicle.com,https://sfchronicle.com/opinion/letterstotheed...,letters: how the country can respond to nashvi...,"enough with the "" thoughts and prayers. "" thre...",Left
8092,97848056,918.0,23-03-29,US,bostonglobe.com,https://bostonglobe.com/2023/03/28/opinion/abo...,the latest antiabortion tactic: asserting the ...,"antiabortion activist trooper elwonger, 25, fe...",Left
8093,90760423,1224.0,23-03-31,US,The Daily Beast,https://www.thedailybeast.com/satan-wants-you-...,inside the horrific (contested) abuse story th...,how did one discredited biography ignite one o...,Left


In [44]:
import pandas as pd
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-mpnet-base-v2")

# Take a subset of the DataFrame
subset = articles  # Select the first 3 rows

# Define a function to count words
def count_words(text):
    return len(text.split())  # Split the text by spaces and count

# Define a function to count tokens
def count_tokens(text):
    tokens = tokenizer.tokenize(text)  # Tokenize the text
    return len(tokens)  # Return the number of tokens

# Apply the functions to calculate word and token counts
subset["word_count"] = subset["body"].apply(count_words)
subset["token_count"] = subset["body"].apply(count_tokens)

# Create a new DataFrame with word counts and token counts
word_token_counts_df = subset[["word_count", "token_count"]]

# Display the subset and word/token counts DataFrame
#print("Subset DataFrame with Counts:")
#print(subset)

print("\nWord and Token Counts DataFrame:")
word_token_counts_df


Token indices sequence length is longer than the specified maximum sequence length for this model (872 > 512). Running this sequence through the model will result in indexing errors



Word and Token Counts DataFrame:


,word_count,token_count
0,681,872
1,1180,1505
2,1854,2346
3,1464,1812
4,2513,3260
...,...,...
8090,1216,1504
8091,628,781
8092,838,1066
8093,1172,1499


In [45]:
# Define the file path where you want to save the CSV
output_file_path = "/work/LauraSørineVoldgaard#8191/data/word_token_counts_df.csv"

# Save the filtered DataFrame as a CSV file
word_token_counts_df.to_csv(output_file_path, index=False, encoding='utf-8')

## Tests - the truncation only happens once you start to encode (i.e. you can tokenize sequences longer than 384 tokens)

### MPNet

In [34]:
from transformers import AutoTokenizer

# Install the tokenizer
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-mpnet-base-v2")

# Example long text
long_text = "This is a very long article. " * 100  # Simulate a long article

# Tokenize the text
tokens = tokenizer.tokenize(long_text)

# Count the number of tokens
print(f"Number of tokens: {len(tokens)}")

# View the tokens
print("Tokens:", tokens[:20])  # Show the first 20 tokens for inspection


Token indices sequence length is longer than the specified maximum sequence length for this model (700 > 512). Running this sequence through the model will result in indexing errors


Number of tokens: 700
Tokens: ['this', 'is', 'a', 'very', 'long', 'article', '.', 'this', 'is', 'a', 'very', 'long', 'article', '.', 'this', 'is', 'a', 'very', 'long', 'article']


In [18]:
encoded = tokenizer(
    long_text,
    max_length=384,  # Set the limit (e.g., 384 or 512)
    truncation=True,  # Enforce truncation
    return_tensors="pt"  # Return as PyTorch tensors
)

# Print the number of tokens after truncation
print(f"Number of tokens after truncation: {encoded['input_ids'].shape[1]}")  # Should print 384

Number of tokens after truncation: 352


In [5]:
text = "Unbelievable things are happening!"
words = text.split()  # Split text by spaces to get word count
tokens = tokenizer.tokenize(text)  # Tokenize text

print(f"Words: {len(words)}")  # Number of words
print(f"Tokens: {len(tokens)}")  # Number of tokens


Words: 4
Tokens: 5
Tokens: ['unbelievable', 'things', 'are', 'happening', '!']


### DeBERTa

In [26]:
# install🔥
%pip install transformers
%pip install torch
%pip install tf-keras # install this because keras needs to work on a previous version of python
%pip install scipy
%pip install pandas
%pip install tensorflow
%pip install -U sentence-transformers
%pip install numpy
%pip install sentencepiece


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.0 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 MB 27.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 22.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 49.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 61.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 16.1 MB/s eta 0:00:00:00:01
  Attempting uninstall: numpy
    Found exist

In [30]:
# import🔥
import tensorflow
from transformers import BertTokenizer, BertModel, TFBertModel # for BERT
from transformers import AutoTokenizer, AutoModel # for DeBERTa
from transformers import DebertaV2Tokenizer, AutoModel # does something but need sentencepiece
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from sentence_transformers import SentenceTransformer
import torch
import sentencepiece

AttributeError: module 'numpy' has no attribute '_no_nep50_warning'

In [32]:
from transformers import DebertaV2Tokenizer
import sentencepiece

# Load the tokenizer
tokenizer = DebertaV2Tokenizer.from_pretrained('microsoft/deberta-v3-large')

# Sample text
text = "This is an example of how DeBERTa handles tokenization."

# Tokenize the text
tokens = tokenizer.tokenize(text)
print("Tokens:", tokens)

# Encode the text (includes converting tokens to input IDs)
encoded = tokenizer.encode_plus(
    text,
    max_length=512,  # Set the maximum sequence length
    truncation=True,  # Truncate if the input exceeds max_length
    padding="max_length",  # Pad to max_length if the input is shorter
    return_tensors="pt"  # Return PyTorch tensors
)

print("Input IDs:", encoded["input_ids"])
print("Attention Mask:", encoded["attention_mask"])


ImportError: 
DebertaV2Tokenizer requires the SentencePiece library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.
